In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [12]:
import pandas as pd
df=pd.read_csv('spam.csv')
df.head(4)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...


In [13]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [14]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [15]:
#Down-sampling
df_ham=df[df['Category']=='ham']
df_spam=df[df['Category']=='spam']
print(df_ham.shape,df_spam.shape)

(4825, 2) (747, 2)


In [16]:
df_ham_down=df_ham.sample(df_spam.shape[0])

In [17]:
df_balanced=pd.concat([df_ham_down,df_spam])
df_balanced.shape

(1494, 2)

In [18]:
df_balanced['Category'].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [19]:
df_balanced.sample(4)

,Category,Message
5487,spam,2p per min to call Germany 08448350055 from yo...
4045,ham,"Yes, my reg is Ciao!"
567,ham,Oooh bed ridden ey? What are YOU thinking of?
2021,ham,Anything lor but toa payoh got place 2 walk me...


In [20]:
df_balanced['spam']=df_balanced['Category'].apply(
    lambda x: 1 if x=='spam' else 0) 
df_balanced.drop('Category',axis=1, inplace=True)

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(
    df_balanced['Message'],df_balanced['spam'],stratify=df_balanced['spam'])

In [22]:
X_train.head(3)

2465    They will pick up and drop in car.so no problem..
3065                           Ok. How many should i buy.
1045    We know someone who you know that fancies you....
Name: Message, dtype: object

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2023-03-14 14:35:27.784958: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-14 14:35:27.784975: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-14 14:35:27.785008: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (professor-dingy.maths.ox.ac.uk): /proc/driver/nvidia/version does not exist
2023-03-14 14:35:27.785248: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def get_sentence_embedding(sentences):
    preprocessed_text=bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)["pooled_output"]
get_sentence_embedding(
    [
    "500$ discount. hurry up",
    "Jihwan, are you up for a baseball game tomorrow?"
    ])

In [ ]:
e = get_sentence_embedding([
    "banana",
    'grapes',
    'mango',
    'Jeff Bezos',
    'Elon Musk'])
from sklearn.metrics.pairwise import cosine_similarity
print('banana and graphes are similar: ', 
      cosine_similarity(e[0],e[1]))
print('Jeff Bezos and Elon Musk are similar: ', 
      cosine_similarity(e[3],e[4]))

In [ ]:
#input layer
text_input = tf.keras.layers.Input(
    shape=(), dtype=tf.string, name='text')
#pass it to bert preprocess
preprocessed_text=bert_preprocess(text_input)
#encoder
outputs=bert_encoder(preprocessed_text)
#neural network layer
l = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
#probability
l = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)
model = Model(inputs = [text_input], output = [l])

In [ ]:
model.summary()

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
    tf.keras.metrics.Precision(name = 'precision'),
    tf.keras.metrics.Recall(name = 'recall')
]
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics=METRICS)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_predict = model.predict(X_test)
y_predict = y_predict.flatten()

In [ ]:
import numpy as np
#Convert sigmoid to 1,0
y_predict = np.where(y_predict > 0.5 , 1, 0)
y_predict

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test,y_predict)
cm

In [ ]:
from matplotlib.pyplot as plt
import seaborn as sns
sns.heatmap(cm, annot=True, fmt = 'd')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test,y_predict))

In [ ]:
reviews = [
    'Enter a chance to win $500!',
    'Good moring Jihwan :)',
    'I got a result!'
]
model.predict(reviews)